## Summary from large execution:
|table_name|Written|Dropped|Total Rows Processed|
|---|---|---|---|
|bpm_bronze|9960852|N/A|9960852|
|bpm_silver|7511287|791619|8302906|
|bpm_quarantine|950317|9010535|9960852|

### Baseline Bronze Rows

In [0]:
describe history hive_metastore.labuser6737320_1740559720_5q94_da_adewd_streaming_etl.bpm_bronze


In [0]:
SELECT count(*), count(distinct device_id, time)
FROM
hive_metastore.labuser6737320_1740559720_5q94_da_adewd_streaming_etl.bpm_bronze
version AS OF 3 
--3:10840345/9034893
--2:  879493/731987
--1:  417164/347116

In [0]:
SELECT 9034893 - 731987 as bpm_silver_appends

In [0]:
SELECT 10840345 - 879493 as bpm_bronze_appends_v3

### Distinct records sent to `bpm_silver` based on `dropDuplicates` in DLT

In [0]:
%python
import pyspark.sql.functions as F
(spark.read.option("versionAsOf", "1").table("hive_metastore.labuser6737320_1740559720_5q94_da_adewd_streaming_etl.bpm_bronze").select("*", F.when(F.col("heartrate") <= 0, "Negative BPM").otherwise("OK").alias("bpm_check")).dropDuplicates(["device_id", "time"])).count()

#3:9034893
#2:731987
#1:347116

### Count Distinct difference between `bpm_silver` versions

In [0]:
select 9034893-731987 as distinct_rows_to_bpm_silver_from_version_delta

In [0]:
SELECT 7511287 + 791619 as distinct_rows_to_bpm_silver_from_expectation_log

In [0]:
%python
rules = {
  "valid_heartrate": "heartrate IS NOT NULL",
  "valid_device_id": "device_id IS NOT NULL",
  "valid_device_id_range": "device_id > 110000"
}

In [0]:
%python
print(f"({' AND '.join(rules.values())})")

In [0]:
SELECT COUNT(*) FROM hive_metastore.labuser6737320_1740559720_5q94_da_adewd_streaming_etl.bpm_bronze
VERSION AS OF 2
WHERE heartrate IS NOT NULL AND device_id IS NOT NULL AND device_id > 110000
--3:9807004
--2:796469

In [0]:
SELECT 9807004 - 796469 as valid_rows_from_v3_query

In [0]:
SELECT 950317 + 9010535